<a href="https://colab.research.google.com/github/AlexadeZ17/datathon2022-alex-gerard-laia-joan/blob/master/src/Datathon_FME_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Datathon FME 2022

Idees iaios actuals
- Furgoneta per tractar iaios

Idees iaios futurs